In [1]:
import numpy as np
from inversion import *
from ray_tracer import *
from calibration import *

In [3]:
# ------------------------------------------------------------------
# 1.  TRUE 1-D velocity model  (km s-1) – what we hope to recover
# ------------------------------------------------------------------
true_velocity_model = [
    {'top': 0.0, 'bottom': 2.0,  'velocity': 2.0},
    {'top': 2.0, 'bottom': 6.0,  'velocity': 3.0},
    {'top': 6.0, 'bottom':15.0,  'velocity': 4.0},
]

# ------------------------------------------------------------------
# 2.  “STARTING” model (deliberately biased slow by ~10 %)
# ------------------------------------------------------------------
start_velocity_model = [
    {'top': 0.0, 'bottom': 2.0,  'velocity': 1.8},   #  −10 %
    {'top': 2.0, 'bottom': 6.0,  'velocity': 2.7},   #  −10 %
    {'top': 6.0, 'bottom':15.0,  'velocity': 3.6},   #  −10 %
]

# ------------------------------------------------------------------
# 3.  Station geometry  (five surface + five down-hole)
# ------------------------------------------------------------------
stations = [
    # surface ring
    {'x':  0.0, 'y':  0.0, 'z': 0.0},
    {'x': 10.0, 'y':  0.0, 'z': 0.0},
    {'x':  0.0, 'y': 10.0, 'z': 0.0},
    {'x': 10.0, 'y': 10.0, 'z': 0.0},
    {'x':  5.0, 'y':  5.0, 'z': 0.0},   # borehole collar
    # down-hole receivers
    {'x':  5.0, 'y':  5.0, 'z': 1.0},
    {'x':  5.0, 'y':  5.0, 'z': 3.0},
    {'x':  5.0, 'y':  5.0, 'z': 5.0},
    {'x':  5.0, 'y':  5.0, 'z': 7.0},
    {'x':  5.0, 'y':  5.0, 'z': 9.0},
]

# ------------------------------------------------------------------
# 4.  Three perfectly known explosion events
# ------------------------------------------------------------------
events = [
    {'x': 2.0, 'y': 2.5, 'z': 0.8, 't0': 0.20},   # in layer 1
    {'x': 7.0, 'y': 3.0, 'z': 3.5, 't0': 0.25},   # in layer 2
    {'x': 4.0, 'y': 8.0, 'z': 8.0, 't0': 0.30},   # in layer 3
]

# ------------------------------------------------------------------
# 5.  Build synthetic picks with TRUE model + Gaussian noise
# ------------------------------------------------------------------
np.random.seed(42)
sigma_t = 0.005             # 5 ms timing uncertainty
picks   = {}                # keyed by (event_idx, station_idx)

print("Synthetic ray parameters (true model):")
for ev_idx, ev in enumerate(events):
    for st_idx, sta in enumerate(stations):
        t_calc, _, p = trace_ray(ev, sta, true_velocity_model)
        t_obs = t_calc + ev['t0'] + np.random.normal(0, sigma_t)
        picks[(ev_idx, st_idx)] = t_obs
        if ev_idx == 0:      # just print once for illustration
            print(f"  Sta {st_idx:02d}  p = {p:7.4f} s/km")

# ------------------------------------------------------------------
# 6.  Run velocity-model calibration
# ------------------------------------------------------------------
print("\nInitial layer velocities (km/s):",
      [lay['velocity'] for lay in start_velocity_model])

calibrated_model = calibrate_velocity_model(
    events=events,
    stations=stations,
    picks=picks,
    velocity_model=start_velocity_model,   # updated in place
    max_iter=200,
    damping=0.02,
    tol=1e-5
)

print("\nCalibrated layer velocities (km/s):",
      [lay['velocity'] for lay in calibrated_model])

print("\nTrue layer velocities (km/s):",
      [lay['velocity'] for lay in true_velocity_model])

Synthetic ray parameters (true model):
  Sta 00  p =  0.4851 s/km
  Sta 01  p =  0.4977 s/km
  Sta 02  p =  0.4974 s/km
  Sta 03  p =  0.4987 s/km
  Sta 04  p =  0.4898 s/km
  Sta 05  p =  0.4994 s/km
  Sta 06  p =  0.3155 s/km
  Sta 07  p =  0.2444 s/km
  Sta 08  p =  0.1753 s/km
  Sta 09  p =  0.1317 s/km

Initial layer velocities (km/s): [1.8, 2.7, 3.6]
Iter 01: RMS = 0.314780  |Δs| = 7.34e-02
Iter 02: RMS = 0.007465  |Δs| = 2.18e-03
Iter 03: RMS = 0.004470  |Δs| = 4.83e-04
Iter 04: RMS = 0.004345  |Δs| = 1.25e-04
Iter 05: RMS = 0.004237  |Δs| = 2.35e-04
Iter 06: RMS = 0.004273  |Δs| = 2.40e-04
Iter 07: RMS = 0.004242  |Δs| = 1.53e-04
Iter 08: RMS = 0.004343  |Δs| = 1.88e-04
Iter 09: RMS = 0.004253  |Δs| = 4.30e-05
Iter 10: RMS = 0.004279  |Δs| = 3.11e-05
Iter 11: RMS = 0.004401  |Δs| = 8.28e-05
Iter 12: RMS = 0.004458  |Δs| = 3.47e-04
Iter 13: RMS = 0.004217  |Δs| = 2.21e-04
Iter 14: RMS = 0.004245  |Δs| = 1.59e-04
Iter 15: RMS = 0.004441  |Δs| = 1.06e-04
Iter 16: RMS = 0.004340  |